<a href="https://colab.research.google.com/github/Sebastian310/Modelos-Predictivos/blob/main/BUSIESS_CASE__SEBAS_BINARIO_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importar librerias

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from xgboost import XGBClassifier


In [ ]:
# Importamos base de datos, eliminamos Filas Vacías y revisamos tipos de variables de las columnas

df = pd.read_excel("DataTFM2.xlsx").dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15576 entries, 0 to 15575
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Round                      15576 non-null  int64         
 1   Date                       15576 non-null  datetime64[ns]
 2   Time                       15576 non-null  object        
 3   Team                       15576 non-null  object        
 4   Team_Score                 15576 non-null  int64         
 5   Team_accum_goals           15576 non-null  int64         
 6   Team_accum_goals_conceded  15576 non-null  int64         
 7   team_accum_points          15576 non-null  int64         
 8   Opponent                   15576 non-null  object        
 9   Opponent_Score             15576 non-null  int64         
 10  opp_accum_goals            15576 non-null  int64         
 11  opp_accum_goals_conceded   15576 non-null  int64         
 12  opp_

In [ ]:
# Creamos 5 columnas ( 5 Fechas) para los resultados del equipo local en fechas anteriores (creando variables rezagada) utilizando el mètodo shift.

df['Team_Points(T-1)'] = df.groupby('Team')['Team_Points'].shift(1)
df['Team_Points(T-2)'] = df.groupby('Team')['Team_Points'].shift(2)
df['Team_Points(T-3)'] = df.groupby('Team')['Team_Points'].shift(3)
df['Team_Points(T-4)'] = df.groupby('Team')['Team_Points'].shift(4)
df['Team_Points(T-5)'] = df.groupby('Team')['Team_Points'].shift(5)


In [ ]:
# Creamos Nuevos dataframes (dfhome) y (dfaway) y se resetea el indice para evitar que los nuevos tengan el indice del dataframe original.

dfaway = df[df["Location"]=="Away"].reset_index(drop=True)
dfhome = df[df["Location"]=="Home"].sort_values(by=["Opponent","season","Round"]).reset_index(drop=True)

In [ ]:
# Crear columnas para resultados del oponente en fechas anteriores (variables rezagadas) (esto para anexar columnas a home)

dfhome['Opp_Points(T-1)'] = dfaway['Team_Points(T-1)']
dfhome['Opp_Points(T-2)'] = dfaway['Team_Points(T-2)']
dfhome['Opp_Points(T-3)'] = dfaway['Team_Points(T-3)']
dfhome['Opp_Points(T-4)'] = dfaway['Team_Points(T-4)']
dfhome['Opp_Points(T-5)'] = dfaway['Team_Points(T-5)']

In [ ]:
# Creamos el nuevo dataset con todas las variables, descartando las que no se usarán , renombrando variables y verificando el tipo de las mismas.

newdf = dfhome.sort_values(by=["Date","Round"]).reset_index(drop=True)
col_eliminar = ["Time","Date", "Location","Opponent_Points","Country","Competition","Team_Score","Opponent_Score"]
newdf.drop(col_eliminar, axis=1,inplace=True)
newnames = {"Team_Points": "Result", "team_accum_points":"Team_accum_points","opp_accum_goals":"Opp_accum_goals","opp_accum_points":"Opp_accum_points","season":"Season"}
newdf.rename(columns=newnames, inplace=True)
newdf.info()

In [ ]:
# Binarizo el resultado  de la variable independiente "Result" del nuevo dataframe, dònde pérdida(o) y empate (1) son igual a 0,  y victoria(3) =1

newdf['Result'] = newdf['Result'].replace({0: 0, 1: 0,3:1})

In [ ]:
# Reorganizamos el Datafreme Home

nuevo_orden_columnas = ['Round', 'Team', 'Team_accum_goals',"Team_accum_goals_conceded", 'Team_accum_points', 'Opponent',
                         'Opp_accum_goals',"opp_accum_goals_conceded", 'Opp_accum_points',
                        'Season', 'Team_Points(T-1)', 'Team_Points(T-2)', 'Team_Points(T-3)',
                        'Team_Points(T-4)', 'Team_Points(T-5)', 'Opp_Points(T-1)',
                        'Opp_Points(T-2)', 'Opp_Points(T-3)', 'Opp_Points(T-4)',
                        'Opp_Points(T-5)', 'Result']

newdf = newdf.reindex(columns=nuevo_orden_columnas)

In [ ]:
# Eliminamos  Valores Vacíos en Team_Points(T-5) y Opp_Points(T-5). Teniendo uniformidad en el dataset final a trabajar.

newdf = newdf.dropna(subset=['Team_Points(T-5)', 'Opp_Points(T-5)'])
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7588 entries, 50 to 7787
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Round                      7588 non-null   int64  
 1   Team                       7588 non-null   object 
 2   Team_accum_goals           7588 non-null   int64  
 3   Team_accum_goals_conceded  7588 non-null   int64  
 4   Team_accum_points          7588 non-null   int64  
 5   Opponent                   7588 non-null   object 
 6   Opp_accum_goals            7588 non-null   int64  
 7   opp_accum_goals_conceded   7588 non-null   int64  
 8   Opp_accum_points           7588 non-null   int64  
 9   Season                     7588 non-null   int64  
 10  Team_Points(T-1)           7588 non-null   float64
 11  Team_Points(T-2)           7588 non-null   float64
 12  Team_Points(T-3)           7588 non-null   float64
 13  Team_Points(T-4)           7588 non-null   floa

In [ ]:
## Defino variables dependientes e independientes
x = newdf[["Round", "Opponent", "Team", "Team_accum_goals", "Team_accum_points",
           "Opp_accum_goals","Team_accum_goals_conceded","opp_accum_goals_conceded", "Opp_accum_points", "Team_Points(T-1)", "Team_Points(T-2)",
            "Team_Points(T-3)", "Team_Points(T-4)", "Team_Points(T-5)", "Opp_Points(T-2)",
            "Opp_Points(T-3)", "Opp_Points(T-4)", "Opp_Points(T-5)"]]
x =pd.get_dummies(x)
y = newdf["Result"]
x.info()

In [ ]:
# Dividimos los datos en conjuntos de entrenamiento y prueba 85% 6450

# Entrenamiento  1 = 6424
X_train = x.iloc[:6434]
y_train = y.iloc[:6434]
X_test = x.iloc[6434:7570]
y_test = y.iloc[6434:7570]

# Reentreno  2
X_train2 = x.iloc[:7570]
y_train2 = y.iloc[:7570]
X_bc2 = x.iloc[7570:7576]
y_bc2 = y.iloc[7570:7576]

# Vuelta  3
X_train3 = x.iloc[:7576]
y_train3 = y.iloc[:7576]
X_bc3 = x.iloc[7576:7582]
y_bc3 = y.iloc[7576:7582]

# Vuelta  4
X_train4 = x.iloc[:7582]
y_train4 = y.iloc[:7582]
X_bc4 = x.iloc[7582:]
y_bc4 = y.iloc[7582:]


Modelos Aplicados

In [ ]:
# ■  - RandomForest

# ■ entreno primera vuelta
clf = RandomForestClassifier(n_estimators=120, max_depth=9, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)                                                            # ■ realizo predicciones
print("Precisión del modelo:", accuracy_score(y_test, y_pred))                          # ■ Evalua modelo


Precisión del modelo: 0.6426056338028169


In [ ]:
# ■ Reentreno 2
clf.fit(X_train2, y_train2)                                                             # ■ Reentreno modelo
y_pred2 = clf.predict(X_bc2)                                                          # ■  Realiza las predicciones con los nuevos datos
print("Precisión del modelo después de reentrenar:", accuracy_score(y_bc2, y_pred2))  # ■  Evalua modelo con Datos 2


Precisión del modelo después de reentrenar: 1.0


In [ ]:
# ■ Reentreno 3
clf.fit(X_train3, y_train3)                                                             # ■ Reentreno modelo
y_pred3 = clf.predict(X_bc3)                                                          # ■  Realiza las predicciones con los nuevos datos
print("Precisión del modelo después de reentrenar:", accuracy_score(y_bc3, y_pred3))  # ■  Evalua modelo con Datos 2

Precisión del modelo después de reentrenar: 0.8333333333333334


In [ ]:
# ■ Reentreno 4
clf.fit(X_train4, y_train4)                                                             # ■ Reentreno modelo
y_pred4 = clf.predict(X_bc4)                                                          # ■  Realiza las predicciones con los nuevos datos
print("Precisión del modelo después de reentrenar:", accuracy_score(y_bc4, y_pred4))  # ■  Evalua modelo con Datos 2


Precisión del modelo después de reentrenar: 0.3333333333333333


In [ ]:
# ■  - XG BOOST

# ■ Entreno primera vuelta
model = XGBClassifier(use_label_encoder=False, n_estimators=187, max_depth=8, learning_rate=0.0369984949752035)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)                                                  # ■ realizo predicciones
accuracy = accuracy_score(y_test, y_pred)                                       # ■ Evalúa el modelo
print(f'Precisión del modelo: {accuracy}')                                      # ■ Evalua modelo


Precisión del modelo: 0.6346830985915493


In [ ]:
# ■ Reentreno 2
model.fit(X_train2, y_train2)                                                   # ■ Reentreno modelo
y_pred2 = model.predict(X_bc2)                                                # ■  Realiza las predicciones con los nuevos datos
accuracy2 = accuracy_score(y_bc2, y_pred2)                                    # ■  Evalua modelo con Datos 2
print(f'Precisión del modelo después de reentrenar: {accuracy2}')

Precisión del modelo después de reentrenar: 0.8333333333333334


In [ ]:
# ■ Reentreno 3
model.fit(X_train3, y_train3)                                                   # ■ Reentreno modelo
y_pred3 = model.predict(X_bc3)                                                # ■  Realiza las predicciones con los nuevos datos
accuracy3 = accuracy_score(y_bc3, y_pred3)                                    # ■  Evalua modelo con Datos 2
print(f'Precisión del modelo después de reentrenar: {accuracy3}')

Precisión del modelo después de reentrenar: 0.8333333333333334


In [ ]:
# ■ Reentreno 4
model.fit(X_train4, y_train4)                                                   # ■ Reentreno modelo
y_pred4 = model.predict(X_bc4)                                                # ■  Realiza las predicciones con los nuevos datos
accuracy4 = accuracy_score(y_bc4, y_pred4)                                    # ■  Evalua modelo con Datos 2
print(f'Precisión del modelo después de reentrenar: {accuracy4}')

Precisión del modelo después de reentrenar: 0.3333333333333333
